In [ ]:
# Mengimpor library PyTorch
import torch
from torch import nn

# Mengimpor library HuggingFace Transformers
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoConfig, AdamW, get_linear_schedule_with_warmup

# Mengimpor library lainnya
import pandas as pd
import numpy as np
import random
from sklearn.metrics import f1_score

# Mendefinisikan fungsi untuk melatih model
def train_model(model, train_loader, val_loader, epochs, lr, save_path):
    # Mengatur device yang akan digunakan untuk melatih model, yaitu CPU atau GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Memindahkan model ke device yang telah ditentukan
    model.to(device)

    # Mengatur optimizer yang akan digunakan untuk mengoptimalkan bobot model, yaitu AdamW
    optimizer = AdamW(model.parameters(), lr=lr)

    # Mengatur scheduler yang akan digunakan untuk mengatur learning rate selama proses pelatihan, yaitu linear decay with warmup
    # Scheduler ini akan membuat learning rate meningkat secara linear pada 10% epoch pertama (warmup), lalu menurun secara linear pada 90% epoch sisanya (decay)
    total_steps = len(train_loader) * epochs # Jumlah total langkah yang akan dilakukan selama proses pelatihan
    warmup_steps = int(total_steps * 0.1) # Jumlah langkah untuk fase warmup
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

    # Mengatur loss function yang akan digunakan untuk menghitung seberapa besar kesalahan antara output model dan label sebenarnya, yaitu CrossEntropyLoss
    # Loss function ini akan mengabaikan nilai -100 pada label_ids saat menghitung loss
    loss_function = nn.CrossEntropyLoss(ignore_index=-100)

    # Membuat variabel untuk menyimpan nilai metric terbaik dan epoch terbaik selama proses pelatihan
    best_metric = 0 # Nilai awal metric terbaik
    best_epoch = 0 # Nilai awal epoch terbaik

    # Melakukan iterasi pada setiap epoch
    for epoch in range(epochs):
        # Membuat list kosong untuk menyimpan nilai loss dan metric untuk setiap batch data pelatihan dan validasi
        train_loss = []
        train_metric = []
        val_loss = []
        val_metric = []

        # Mengubah mode model menjadi mode train
        model.train()

        # Melakukan iterasi pada setiap batch data dalam train_loader
        for batch in train_loader:
            # Mengambil input_ids, attention_mask, dan label_ids dari batch data dan memindahkannya ke device yang telah ditentukan
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            label_ids = batch["label_ids"].to(device)

            # Menghitung output dari model dengan menggunakan method forward dan input_ids, attention_mask, dan label_ids
            output = model(input_ids=input_ids, attention_mask=attention_mask, labels=label_ids)

            # Mengambil logits dari output model
            logits = output.logits

            # Menghitung loss dari output dengan menggunakan loss function dan label_ids
            loss = loss_function(logits.view(-1, model.config.num_labels), label_ids.view(-1))

            # Menambahkan nilai loss ke list train_loss
            train_loss.append(loss.item())

            # Melakukan backpropagation dan optimis